# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
!pip install selenium

In [7]:
from selenium import webdriver
# permite enviar as teclas
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# abrir um navegador
navegador = webdriver.Chrome()
navegador.get("https://www.google.com/")

# clicar em uma região e escrever
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
# no mesmo campo, ele irá apertar enter
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# pegar o valor
cotacao_dolar = navegador.find_element(
    By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(
    By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

# cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value').replace(',','.')
print(cotacao_dolar, cotacao_euro, cotacao_ouro)
navegador.quit()

5.099409 5.304145171 298.87


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [55]:
import pandas as pd

dataset = pd.read_excel('Produtos.xlsx')
dataset.head(7)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Trocando as cotações

In [56]:
dataset.loc[dataset['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
dataset.loc[dataset['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
dataset.loc[dataset['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)
display(dataset)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.533100,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.330364,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.533100,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.533100,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.330364,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.533100,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,324.800000,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [57]:
# Preço de Compra = Preço Original * Cotação
dataset['Preço de Compra'] = dataset['Preço Original'] * dataset['Cotação']
# Preço de venda = Preço de compra * Margem
dataset['Preço de Venda'] = dataset['Preço de Compra'] * dataset['Margem']
display(dataset)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.533100,5533.044669,1.40,7746.262537
1,Carro Renault,4500.00,Euro,6.330364,28486.639706,2.00,56973.279411
2,Notebook Dell,899.99,Dólar,5.533100,4979.734669,1.70,8465.548937
3,IPhone,799.00,Dólar,5.533100,4420.946900,1.70,7515.609730
4,Carro Fiat,3000.00,Euro,6.330364,18991.093137,1.90,36083.076960
5,Celular Xiaomi,480.48,Dólar,5.533100,2658.543888,2.00,5317.087776
6,Joia 20g,20.00,Ouro,324.800000,6496.000000,1.15,7470.400000


### Agora vamos exportar a nova base de preços atualizada

- Nome igual ao da base de dados, ele substitui.
- index -> False (não traz os índices do python)
- index -> True (exporta com os índices do python)

In [58]:
dataset.to_excel('Produtos (atualizado).xlsx', index=False)